# Core Question Process
What are the overall trends in sales?
 --> This starts very broad, so we need to iteratively clarify

### Part 1
READ Framework for framing the project before querying anything

R - Representative Data: 
    do we have the right data to answer this? 
We don't have order dates or revenue so we can't see change over calendar periods or true sales volume beyond quantities. We have day of the week, reoder information, even time of day so we can see trends in those areas via order/product counts. 

E - Exec questions: 
    transformation from vague to business-relevant
Metric clarifications: what are we measuring? 
    Before: What are the overall trends in sales?
    After: How did total order volume and reorder rate change?
Dimension: how do we slice it? 
    Department, time of day, day of the week.
    We can remove time of day for now. 
Deliverable: for whom, in what format? 
    Marketing and operations management would likely find insights useful. 
Clarified question: How do total order volume and reorder rate change across departments and between weekdays and weekends? .

A - Analytical Framework: 
    Time series - time of day and day of week
    Segmentation by department

D - Data best practices: 
    Check for nulls, odd ranges


After all this, we have data that may be able to help Marketing & Ops understand order behavior across departments across different days of the week and hours of the day, and modify their strategies accordingly. 

In [2]:

import duckdb
con = duckdb.connect("instacart.duckdb")

df_user_orders = con.execute("""
    SELECT 
    SUM(order_id IS NULL) AS null_id,
    SUM(user_id IS NULL) AS null_user,
    SUM(eval_set IS NULL) AS null_eval,
    SUM(order_number IS NULL) AS null_num,
    SUM(order_dow IS NULL) AS null_dow,
    SUM(order_hour_of_day IS NULL) AS null_hour,
    -- SUM(days_since_prior_order IS NULL) AS null_days_since, -> not counting because this represents the number of first time orders
    MIN(CAST(order_hour_of_day AS INT)) AS min_hour,
    MAX(CAST(order_hour_of_day AS INT)) AS max_hour
    FROM orders;
""").fetchdf()
print(df_user_orders)



   null_id  null_user  null_eval  null_num  null_dow  null_hour  min_hour  \
0      0.0        0.0        0.0       0.0       0.0        0.0         0   

   max_hour  
0        23  


Data is well-cleaned, mostly IDs and integers. 

### Part 2: Mapping goals to data features

Stakeholder Goals  | what KPIs and dimensions matter the most? 
    When are we selling - what can we do to take advantage of that and what can we do to make up for the less efficient times? 

Columns and Coverage  | what data do we have available and how can I use it?   
    Volume of items per department per hour of individual days
    
Aggregates and Anomalies  | the high level metrics, outliers, and unexpected patterns  
| Metric | Why it Matters | Quick SQL |
| :- | :- | :- |
| Total orders | Anchor everything else | SELECT COUNT(*) AS n_orders FROM orders; 
| Avg items per order | Spot exceptionally large baskets later | sql WITH line_ct AS (SELECT order_id, COUNT(*) AS n_items FROM order_products_train GROUP BY order_id) SELECT AVG(n_items) FROM line_ct;
| Overall reorder‑rate | Acts as “global mean” for dept/aisle compare | SELECT AVG(reordered)::DOUBLE AS reorder_rate FROM order_products_train;

Notable Segments  | slice by category, time, or other key dimensions to surface early insights 
    See aggregates by key dimensions below

In [32]:
# Orders by hour of day
orders_by_hour = con.execute("""
    SELECT 
    order_hour_of_day,
    COUNT(*) AS orders
    FROM orders
    group by order_hour_of_day
    order by order_hour_of_day;
""").fetchdf()
print(orders_by_hour)

orders_by_hour_insights = con.execute("""
    SELECT
        COUNT(*) as total,
        AVG(orders) as avg,
        MIN(orders) as min,
        MAX(orders) as max
    FROM
        (SELECT 
        order_hour_of_day,
        COUNT(*) AS orders
        FROM orders
        group by order_hour_of_day
        order by order_hour_of_day) AS orders_per_hour;
""").fetchdf()
print(orders_by_hour_insights)

   order_hour_of_day  orders
0                 00   22758
1                 01   12398
2                 02    7539
3                 03    5474
4                 04    5527
5                 05    9569
6                 06   30529
7                 07   91868
8                 08  178201
9                 09  257812
10                10  288418
11                11  284728
12                12  272841
13                13  277999
14                14  283042
15                15  283639
16                16  272553
17                17  228795
18                18  182912
19                19  140569
20                20  104292
21                21   78109
22                22   61468
23                23   40043
   total         avg   min     max
0     24  142545.125  5474  288418


In [3]:
# Orders by day of week
orders_by_day = con.execute("""
    SELECT 
    COUNT(CASE WHEN order_dow = 0 Then 1 END) AS Sunday,
    COUNT(CASE WHEN order_dow = 1 Then 1 END) AS Monday,
    COUNT(CASE WHEN order_dow = 2 Then 1 END) AS Tuesday,
    COUNT(CASE WHEN order_dow = 3 Then 1 END) AS Wednesday,
    COUNT(CASE WHEN order_dow = 4 Then 1 END) AS Thursday,
    COUNT(CASE WHEN order_dow = 5 Then 1 END) AS Friday,
    COUNT(CASE WHEN order_dow = 6 Then 1 END) AS Saturday,
    FROM orders
    ;
""").fetchdf()
print(orders_by_day)

   Sunday  Monday  Tuesday  Wednesday  Thursday  Friday  Saturday
0  600905  587478   467260     436972    426339  453368    448761


In [19]:
# Product volume & reorder‑rate by department

department_volume = con.execute("""
    SELECT                
        d.department,
        COUNT(*) as product_count,
        AVG(opt.reordered) as reorder_rate
    FROM order_products_train opt
    JOIN products p on opt.product_id = p.product_id
    JOIN departments d on d.department_id = p.department_id
    GROUP BY 1
    ORDER BY 2 DESC
    ;
""").fetchdf()
print(department_volume)

         department  product_count  reorder_rate
0           produce         409087      0.664617
1        dairy eggs         217051      0.674966
2            snacks         118862      0.581363
3         beverages         114046      0.658155
4            frozen         100426      0.559297
5            pantry          81242      0.363088
6            bakery          48394      0.634211
7      canned goods          46799      0.486805
8              deli          44291      0.617891
9   dry goods pasta          38713      0.487821
10        household          35986      0.427166
11     meat seafood          30307      0.590854
12        breakfast          29500      0.571661
13    personal care          21570      0.337089
14           babies          14941      0.541062
15    international          11902      0.379936
16          missing           8251      0.381530
17          alcohol           5598      0.606824
18             pets           4497      0.630198
19            other 

In [30]:
# Items per order insights 

order_items = con.execute("""
    SELECT 
        COUNT(*) as total orders,
        COUNT(CASE WHEN item_count >= 70 THEN 1 END) AS seventy_plus,
        COUNT(CASE WHEN item_count >= 60 AND item_count < 70 THEN 1 END) AS sixty_to_seventy,
        COUNT(CASE WHEN item_count >= 50 AND item_count < 60 THEN 1 END) AS fifty_to_sixty,
        COUNT(CASE WHEN item_count >= 40 AND item_count < 50 THEN 1 END) AS forty_to_fifty,
        COUNT(CASE WHEN item_count >= 30 AND item_count < 40 THEN 1 END) AS thirty_to_forty,
        COUNT(CASE WHEN item_count >= 20 AND item_count < 30 THEN 1 END) AS twenty_to_thirty,
        COUNT(CASE WHEN item_count >= 10 AND item_count < 20 THEN 1 END) AS ten_to_twenty,
        COUNT(CASE WHEN item_count < 10 THEN 1 END) AS under_ten,             
        AVG(item_count) as average,
        Min(item_count) as min,
        Max(item_count) as max
    FROM 
        (SELECT
        order_id,
        COUNT(*) AS item_count
        FROM order_products_train
        GROUP BY order_id) AS item_counts
    ;
""").fetchdf()
print(order_items)

   totals  seventy_plus  sixty_to_seventy  fifty_to_sixty  forty_to_fifty  \
0  131209            14                31             148             650   

   thirty_to_forty  twenty_to_thirty  ten_to_twenty  under_ten    average  \
0             3066             12632          42860      71808  10.552759   

   min  max  
0    1   80  


In [49]:
# Any significance in when the larger orders are happening? Anything above 50 items

order_items = con.execute("""
    SELECT 
        item_counts_outliers.order_id,
        item_counts_outliers.item_count,
        o.order_dow,
        o.order_hour_of_day
    FROM
        (SELECT
            opt.order_id,
            COUNT(*) AS item_count, 
        FROM order_products_train opt
        GROUP BY opt.order_id
        HAVING item_count >= 50) 
    AS item_counts_outliers
    JOIN orders o on item_counts_outliers.order_id = o.order_id
    ;
""").fetchdf()
print(order_items)

order_items = con.execute("""
    SELECT 
        order_dow,
        COUNT(*) AS dow_counts
    FROM
        (SELECT 
            item_counts_outliers.order_id,
            item_counts_outliers.item_count,
            o.order_dow,
            o.order_hour_of_day
        FROM
            (SELECT
                opt.order_id,
                COUNT(*) AS item_count, 
            FROM order_products_train opt
            GROUP BY opt.order_id
            HAVING item_count >= 50) 
        AS item_counts_outliers
        JOIN orders o on item_counts_outliers.order_id = o.order_id) AS c
    GROUP BY order_dow
    ;
""").fetchdf()
print(order_items)

order_items = con.execute("""
    SELECT 
        order_hour_of_day,
        COUNT(*) AS hod_counts
    FROM
        (SELECT 
            item_counts_outliers.order_id,
            item_counts_outliers.item_count,
            o.order_dow,
            o.order_hour_of_day
        FROM
            (SELECT
                opt.order_id,
                COUNT(*) AS item_count, 
            FROM order_products_train opt
            GROUP BY opt.order_id
            HAVING item_count >= 50) 
        AS item_counts_outliers
        JOIN orders o on item_counts_outliers.order_id = o.order_id) AS c
    GROUP BY order_hour_of_day
    ;
""").fetchdf()
print(order_items)

     order_id  item_count  order_dow order_hour_of_day
0     1016866          57          4                16
1     2232869          50          1                15
2     2823840          61          3                13
3     1100193          55          5                16
4      319031          52          2                13
..        ...         ...        ...               ...
188   2658620          50          3                03
189   1997174          53          6                11
190    690200          53          5                12
191   2803519          52          6                11
192    653280          72          0                21

[193 rows x 4 columns]
   order_dow  dow_counts
0          0          41
1          1          33
2          2          24
3          3          23
4          4          19
5          5          25
6          6          28
   order_hour_of_day  hod_counts
0                 12          11
1                 22           7
2                

Compare outlier hod and dow counts in proportion to totals.

In [72]:
# compare orders by day of week to outliers - larger proportions on any days? 

order_items = con.execute("""
    SELECT
        CASE WHEN order_dow = 0 THEN 'SUNDAY'
            WHEN order_dow = 1 THEN 'MONDAY'
            WHEN order_dow = 2 THEN 'TUESDAY'
            WHEN order_dow = 3 THEN 'WEDNESDAY'
            WHEN order_dow = 4 THEN 'THURSDAY'
            WHEN order_dow = 5 THEN 'FRIDAY'
            WHEN order_dow = 6 THEN 'SATURDAY'
        END AS  order_dow_name,
        SUM(is_outlier)::INT AS outlier_orders,
        COUNT(*) AS total_orders,
        ROUND(SUM(is_outlier)/COUNT(*),4) AS outlier_rate
    FROM 
        (SELECT
            o.order_dow,
            CASE WHEN ic.item_count >= 50 THEN 1 ELSE 0 END AS is_outlier
        FROM (
            SELECT
                opt.order_id,
                COUNT(*) AS item_count 
            FROM order_products_train AS opt
            GROUP BY opt.order_id) 
        as ic
        JOIN orders o  ON ic.order_id = o.order_id) AS labeled_orders
    GROUP BY order_dow
    ORDER BY order_dow;
""").fetchdf()
print(order_items)

  order_dow_name  outlier_orders  total_orders  outlier_rate
0         SUNDAY              41         27465        0.0015
1         MONDAY              33         19672        0.0017
2        TUESDAY              24         16119        0.0015
3      WEDNESDAY              23         15687        0.0015
4       THURSDAY              19         15959        0.0012
5         FRIDAY              25         17406        0.0014
6       SATURDAY              28         18901        0.0015


Slightly higher outlier rates on Monday, slightly lower on Thursday, but no surprises or significance. 

## Insights

### 1. Peak ordering happens late morning to midafternoon
Users probably place orders when meal planning, perhaps when getting ready to return from work or while thinking about lunch. Promos and staffing should follow this curve. Incentives to make sure there are enough shoppers on hand. 
Schedule notifications and ads soon before and throughout the 10AM-3pm window. 

### 2. Sunday is most popular day
Weekend uptick in demand is real and makes sense, but Saturday significantly underperforming Sunday could indicate room for improvement via targeting Saturdays. 

### 3. Most carts are very small (<10 items)
We can call it most users being in "top-up" mode, just getting a few necessities. 
Over 50% of orders are <10 items. Average, even with outliers up to 80 items, is just over 10 items per cart. 
Significant cross-sell opportunity with correct product placement and 1-click adds. 

### 4. Departmental differences seem to reflect normal consumer habits
Higher reorder rates in deli, produce, alcohol, etc. make sense. Lower reorder rates in personal care/household COULD indicate stocking issues but are also in line with expectations for that kind of product. Fresh produce is needed every week. Household items likely not. No significant concern, but worth investigating potential for upping reorders in less popular depts. 

### 5. Large orders cluster in later afternoon. 
Orders over 50 items are less than 0.05%, but they are usually between 4-7pm. 
Small businesses stocking up late could make up a lot of this business - could explore opportunities for partnerships there. 

## Specific Insights Table
| Metric / Dimension | Finding | Team |
| :- | :- | :- |
| Hour of Day | Orders peak 10-15hr, 2 stdev above mean at peak | Ops/Marketing |
| Day of Week | Sunday highest, Tues-Thurs lowest | Ops/Marketing | 
| Department reorder rates | Produce / deli / alcohol higher than global reoder rate, personal care and home low | Product, ops |
| Cart size disstribution | 55%+ <10 items, avg 10.6, long tail to 80 max | ops, growth(?) |
| Order size by hour | few orders over 50 items, but they are clustered in late afternoon 16-19hr | Operations | 